<a href="https://colab.research.google.com/github/alejandraberbesi/LLMs/blob/main/Intro_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# init

In [50]:
from huggingface_hub import login
from google.colab import userdata

HF_TOKEN = userdata.get("HF_TOKEN")  # Fetches the token from Colab Secrets
login(HF_TOKEN)

In [51]:
from transformers.utils import logging

logging.set_verbosity_error()  # Suppress progress bars and unnecessary logs

# Pipelines with hugging face

In [52]:
from transformers import pipeline

In [53]:
summarizer = pipeline(task="summarization", model="cnicu/t5-small-booksum") #model used for summarization

text = """
Artificial Intelligence (AI) is transforming various industries by enabling machines to perform tasks
that typically require human intelligence. These tasks include natural language processing, computer
vision, speech recognition, and decision-making. AI models, particularly deep learning models, are
trained on large datasets to recognize patterns and make predictions. In recent years, advancements in
AI have led to applications such as self-driving cars, medical diagnosis, and chatbots. Despite its
benefits, AI also raises ethical concerns, including bias in decision-making and the impact on employment.
As AI continues to evolve, researchers and policymakers must address these challenges to ensure
responsible AI development.
"""

summary = summarizer(text,
                     max_length=50 #limit response to 50 tokens
                     )

print(summary[0]["summary_text"])

AI models, particularly deep learning models, are trained on large datasets to recognize patterns and make predictions. In recent years, advancements in AI have led to applications such as self-driving cars, medical diagnosis, and chatbots


In [54]:
generator = pipeline("text-generation", model="gpt2")  # Model used for text generation

# Example customer complaint
customer_complaint = "The product was horrible, and the service was very slow."

# Predefined apology response
apology_response = "Dear client, we sincerely apologize for the inconvenience you reported"

# Create the prompt with structured input, this is how the response starts
prompt = f"""Customer complaint: {customer_complaint}.\nCompany's response to the customer: {apology_response}.  """

# Generate a response using the model
outputs = generator(
    prompt,
    max_length=150,  # Limit response to 150 tokens
    pad_token_id=generator.tokenizer.eos_token_id,  # Use EOS token to avoid padding issues (GPT-2 lacks a default pad token)
    truncation=True  # Truncate input if it's too long
)

# Print the generated text
print(outputs[0]["generated_text"])  # Extract and display the generated response

Customer complaint: The product was horrible, and the service was very slow..
Company's response to the customer: Dear client, we sincerely apologize for the inconvenience you reported.  ***************************
To be honest, the customer contacted me about four times. They have never contacted us about this and I really don't know what could have caused the customers problems. ***************************
To us customers this could not be your fault, the complaint. It appears that these three products were purchased at the same prices as two other packages, which resulted in total delivery costs for consumers. While it would be hard to find good prices, the price tag is not outrageous at all. In fact, I cannot see any way for us to claim,


In [55]:
spanish_text = "La inteligencia artificial esta cambiando el planeta"

translator = pipeline(task="translation_es_to_en", model="Helsinki-NLP/opus-mt-es-en")

translations = translator(spanish_text,
                          clean_up_tokenization_spaces=True #Fixes spacing issues
                          )

print(translations[0]["translation_text"])

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Artificial intelligence is changing the planet.


## transformers architecture

🔍 **What’s Different in Traditional NLP terms vs LLMs?**

1️⃣ Encoding in Traditional NLP = Simple static embeddings (Word2Vec, TF-IDF).

1️⃣.1️⃣ Encoding in LLMs = Context-aware representations (transformers track relationships between words).

2️⃣ Decoding in Traditional NLP = Retrieving stored text or using basic grammar rules.

2️⃣.1️⃣Decoding in LLMs = Generating new text using self-attention and autoregressive prediction.


| **Model Type**      | **Description**                                             | **Use Cases**                                      | **Example Models**               |
|---------------------|------------------------------------------------|-------------------------------------------------|--------------------------------|
| **Encoder**      | Reads and **understands** text, but doesn't generate new text | Classification, Sentiment Analysis, Named Entity Recognition (NER) | **BERT, RoBERTa, DistilBERT, ALBERT** |
| **Decoder**    | **Generates** text based on input (auto-regressive)  | Text Completion, Chatbots, Story Writing | **GPT-2, GPT-3, LLaMA, Falcon** |
| **Encoder-Decoder**  | First understands the input (encoder), then generates **transformed output** (decoder) | Summarization, Translation, Question Answering | **T5, BART, mBART, FLAN-T5** |


In [56]:
def identify_llm_architecture(model_pipeline):

    # Get the model architecture
    model = model_pipeline.model
    model_type = model.config.architectures[0].lower()  # Get architecture type, and make lowercase

    # Identify architecture based on the model type
    if "bert" in model_type or "roberta" in model_type or "deberta" in model_type:
        print(f"the model arch of {model_type} is Encoder")
    elif "gpt" in model_type or "opt" in model_type or "bloom" in model_type:
        print(f"the model arch of {model_type} is Decoder")
    elif "t5" in model_type or "bart" in model_type or "marian" in model_type:
        print(f"the model arch of {model_type} is Encoder-Decoder")
    else:
        print(f"Other architecture type: {model_type}")

In [57]:
summarizer.model.config.architectures

['T5ForConditionalGeneration']

In [58]:
identify_llm_architecture(summarizer)

the model arch of t5forconditionalgeneration is Encoder-Decoder


In [59]:
identify_llm_architecture(generator)

the model arch of gpt2lmheadmodel is Decoder


In [60]:
identify_llm_architecture(translator)

the model arch of marianmtmodel is Encoder-Decoder


In [72]:
#  extractive question-answering.
text= """
Bogotá is the capital of and largest city in Colombia. It is a place of convergence for people from all around the country and is therefore diverse and multicultural. Within this city, the past and present come together.
The city is also very green thanks to its parks and the hills that extend along its eastern boundary, dwarfed by their two highest points, Monserrate and Guadalupe. The landscape that the people of Bogotá enjoy daily—the sea of green that makes up the Andes mountain range, rising up in the east—would be nearly impossible to find in any other large city.
But Bogotá also has the colors of modern buildings and authentic colonial roof tiles that are a true historical treasure from the colonial era.
Thanks to this fusion of the past and present, Bogotá is an ideal location with history, entertainment, delicious cuisine, culture, business, and much more.
"""

question="what are the characteristics of bogota?"

qa = pipeline(task="question-answering", model="distilbert-base-uncased-distilled-squad")

output = qa(question=question, context=text)

print(output['answer'])
print("----------------")
identify_llm_architecture(qa)

history, entertainment, delicious cuisine, culture, business, and much more
----------------
the model arch of distilbertforquestionanswering is Encoder


In [73]:
#  generative question-answering

qa2 = pipeline(task="text-generation", model="gpt2")

input_text = f"Context: {text}\n\nQuestion: {question}\n\nAnswer:"

output = qa2(input_text,
             max_new_tokens=50 # controls only the number of tokens generated by the model, not the input length
             )

print(output[0]['generated_text'])
print("----------------")
identify_llm_architecture(qa2)

Context: 
Bogotá is the capital of and largest city in Colombia. It is a place of convergence for people from all around the country and is therefore diverse and multicultural. Within this city, the past and present come together.
The city is also very green thanks to its parks and the hills that extend along its eastern boundary, dwarfed by their two highest points, Monserrate and Guadalupe. The landscape that the people of Bogotá enjoy daily—the sea of green that makes up the Andes mountain range, rising up in the east—would be nearly impossible to find in any other large city.
But Bogotá also has the colors of modern buildings and authentic colonial roof tiles that are a true historical treasure from the colonial era.
Thanks to this fusion of the past and present, Bogotá is an ideal location with history, entertainment, delicious cuisine, culture, business, and much more.


Question: what are the characteristics of bogota?

Answer: the city contains a huge cultural tradition but at 